## 参考文档

1. [Doc2Vec Tutorial on the Lee Dataset](https://github.com/RaReTechnologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb)
2. [Tutorial 1: Corpora and Vector Spaces](https://github.com/RaReTechnologies/gensim/blob/develop/docs/notebooks/Corpora_and_Vector_Spaces.ipynb)

## 计划

参考[关于 corpus 格式的转换（与 scipy 的转换）](https://render.githubusercontent.com/view/ipynb?commit=d4c39026837770710d336849193822d032d3ad99&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f526152652d546563686e6f6c6f676965732f67656e73696d2f643463333930323638333737373037313064333336383439313933383232643033326433616439392f646f63732f6e6f7465626f6f6b732f436f72706f72615f616e645f566563746f725f5370616365732e6970796e62&nwo=RaReTechnologies%2Fgensim&path=docs%2Fnotebooks%2FCorpora_and_Vector_Spaces.ipynb&repository_id=1349775#CorpusFormats)，再参考gensim
[关于如何建立 Doc2Vec 的教程之一](https://github.com/RaReTechnologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb)，拟定计划为：

1. 读取经过预处理的文件（此前已经被 newfil.pl 处理过，并保存到合适的模型中
2. [从文本建模](https://render.githubusercontent.com/view/ipynb?commit=d4c39026837770710d336849193822d032d3ad99&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f526152652d546563686e6f6c6f676965732f67656e73696d2f643463333930323638333737373037313064333336383439313933383232643033326433616439392f646f63732f6e6f7465626f6f6b732f646f63327665632d6c65652e6970796e62&nwo=RaReTechnologies%2Fgensim&path=docs%2Fnotebooks%2Fdoc2veclee.ipynb&repository_id=1349775#Training-the-Model)
3. 根据已经建立好的模型，对训练数据和测试数据[推导](https://render.githubusercontent.com/view/ipynb?commit=d4c39026837770710d336849193822d032d3ad99&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f526152652d546563686e6f6c6f676965732f67656e73696d2f643463333930323638333737373037313064333336383439313933383232643033326433616439392f646f63732f6e6f7465626f6f6b732f646f63327665632d6c65652e6970796e62&nwo=RaReTechnologies%2Fgensim&path=docs%2Fnotebooks%2Fdoc2veclee.ipynb&repository_id=1349775#Inferring-a-Vector)出向量表示（Doc2Vec
表示）
4. 处理步骤 3. 中的向量表示，保存到 DataFrame 中，使之能被 scikit-learn 接受
5. 使用 scikit-learn 进行训练
6. 评估训练好的分类器
7. 如有余力，尝试 (Word2Vec + Mean) 或 (Word2Vec + Clustering) 的方法

## 以下是 Word2Vec 部分

In [1]:
# Step 0: Import modules

import gensim
import os
import numpy as np
import pandas as pd
import random
import pandas as pd
import numpy as np
from IPython.display import display
%matplotlib inline

print "Step 0 succeed."

Step 0 succeed.


In [2]:
text8str = ""
with open(os.path.join(os.getcwd(), 'text8'), 'r') as readf:
    text8str = readf.read()

text8 = text8str.split()#gensim.corpora.dictionary.Dictionary([text8str.split()])
print len(text8)

vecsize = 784
sentences = gensim.models.word2vec.Text8Corpus('text8')

print "get sentences from text8 successfully"

17005207
get sentences from text8 successfully


In [4]:
model = gensim.models.word2vec.Word2Vec(sentences=sentences, iter=10, size=vecsize, sg=1, window=3, workers=4, min_count=1)

print "Model created. See some test:"

print model.wv.most_similar(positive=['king', 'woman'], topn=1)

Model created. See some test:
[(u'olympias', 0.5039260387420654)]


In [16]:
# # Step 6: Visualize the embeddings.


# def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
#   assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
#   plt.figure(figsize=(18, 18))  # in inches
#   for i, label in enumerate(labels):
#     x, y = low_dim_embs[i, :]
#     plt.scatter(x, y)
#     plt.annotate(label,
#                  xy=(x, y),
#                  xytext=(5, 2),
#                  textcoords='offset points',
#                  ha='right',
#                  va='bottom')

#   plt.savefig(filename)

# from sklearn.manifold import TSNE

# tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
# plot_only = 500
# low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
# labels = [reverse_dictionary[i] for i in xrange(plot_only)]
# plot_with_labels(low_dim_embs, labels)

print model.wv[]

SyntaxError: invalid syntax (<ipython-input-16-4d6fd86df144>, line 27)

In [6]:
# update vocabulary using  training data

pathtmp = os.path.join(os.getcwd(), 'trialdata', 'sentences-train')
sentencesBuffer = []
with open(pathtmp, 'r') as f:
    buff = f.read()
    sentencesBuffer = buff.split('\n')
model.build_vocab(sentencesBuffer, update=True)

print "update vocabulary from training data successfully"

update vocabulary from training data successfully


In [8]:
# save model
path_model = os.path.join(os.getcwd(), 'models', 'gensim.word2vec')
model.save(path_model)

print "save model finished"

save model finished


In [10]:
# Step 1: preprocess the data
paths = {}
paths['dir.train'] = os.path.join(os.getcwd(), 'trialdata', 'train')
paths['dir.test'] = os.path.join(os.getcwd(), 'trialdata', 'test')

stopwords = ""
with open('stoplist.txt', 'r') as stoplistfile:
    stopwords = stoplistfile.read()
stopwords = stopwords.split()
with open('stoplist2.txt', 'r') as stoplistfile2:
    stopwords2 = stoplistfile2.read()
    stopwords2 = stopwords2.split('\n')
    stopwords = set(stopwords)
    stopwords = list(stopwords.union(set(stopwords)))

# vocabList = model.wv.vocab.items()[:200]

print "Step 1 Succeed"

Step 1 Succeed


In [11]:
# Step 2: read data and save it in data['vec.train'] 和 data['vec.test']

data = {}
data['vec.train'] = {'w2v.mean':[], 'class':[]}
data['vec.test'] = {'w2v.mean':[], 'class':[]}

for tpart in ['train', 'test']:
    dirpath = paths['dir.{}'.format(tpart)]
    for (ind, cls) in enumerate(os.listdir(dirpath)):
        clspath = os.path.join(dirpath, cls)
        files = os.listdir(clspath)
        for f in files:
            fpath = os.path.join(clspath, f)
            with open(fpath, 'r') as readf:
                tokens = [token for token in readf.read().split() if token not in stopwords]#readf.read().split()#
                # Word2Vec representation
                # begin
                vec = np.array([0.0 for i in range(vecsize)])
                expectationVal = np.array([0.0 for i in range(vecsize)])
                for token in tokens:
                    try:
                        vec = model[token]
                    except:
                        vec += expectationVal
                vec = vec / float(len(tokens))
                 # end
            data['vec.{}'.format(tpart)]['w2v.mean'].append(vec)
            data['vec.{}'.format(tpart)]['class'].append(cls)

    tmp = data['vec.{}'.format(tpart)]
    ind = (random.sample(range(len(tmp['class'])), 1))[0]
    print "sample(transformed) from {}[{}]:\n[corpus]\n {}\n[class]\n{}".format(tpart, ind, tmp['w2v.mean'][ind], tmp['class'][ind])
    print
    
print "Step 2 Succeed"

sample(transformed) from train[476]:
[corpus]
 [  2.01418949e-03   8.79096915e-04   9.63741390e-04   4.49759047e-03
  -9.66929772e-04  -9.44941770e-04   2.49703444e-04   3.28460964e-03
   1.53567153e-03  -1.11198029e-03  -1.46805239e-03  -5.08307060e-03
  -8.01076088e-03  -3.18402541e-03   4.93063731e-03   6.99899020e-03
   1.70246861e-03  -6.15934934e-03  -7.33983424e-03   2.54932721e-03
  -4.26344806e-03   4.23420686e-03   6.52407063e-03   8.21447279e-03
   5.95517084e-03   3.20885284e-03   3.74810048e-03   5.35540981e-03
  -9.11107194e-03  -1.22550447e-02   6.19579665e-03   7.43819308e-03
   1.72077795e-03  -4.61357366e-03  -1.57011987e-03   2.73851882e-04
  -5.32102014e-04   3.90591030e-03   9.78282839e-03  -9.93143301e-03
  -4.48459154e-03   1.13163912e-03  -1.15403917e-03  -3.71531863e-03
  -2.31980672e-03  -6.44419901e-03  -4.05846536e-03  -2.30461429e-03
  -1.96479214e-03  -2.82171508e-03  -3.41632956e-04  -1.13052854e-04
  -8.33141617e-03   8.11964448e-04   5.43464348e-03  -2.

sample(transformed) from test[1093]:
[corpus]
 [ -7.36622559e-03  -1.89325586e-03   1.70940335e-03  -3.66958580e-03
  -1.96583150e-03   4.26712306e-03   1.41670008e-03   1.11577064e-02
   2.67757539e-04  -1.29008712e-02  -2.06383807e-03   8.32246430e-03
   1.18560372e-02  -5.43810846e-03   5.42876357e-03  -1.04657537e-03
   4.05287463e-03  -6.62509771e-03   4.94791940e-03  -3.57391487e-04
  -3.41453752e-03  -3.44703160e-03  -6.32809335e-03  -3.13306623e-03
  -6.66267332e-03   6.60281582e-03  -4.72960714e-03   5.24853473e-04
   7.32952962e-03  -4.85643250e-04   6.89084129e-03  -6.32066140e-03
   1.04813064e-02   2.40016961e-03  -6.75951224e-03  -8.41117871e-04
  -4.29141102e-03  -6.40991936e-03  -3.42603959e-03  -3.78411962e-03
   4.99478541e-04  -7.61676021e-03  -6.26896974e-04  -4.73098073e-04
   1.80987641e-03   4.27909108e-04   1.35039189e-03   7.38885545e-04
   2.28560762e-03   6.28045294e-03   3.50345322e-03  -8.18396802e-04
  -2.05867924e-04   9.38844925e-04   2.01890711e-03   5.

In [12]:
# Step 3: Save in Pandas.DataFrame
#
# 将 data['matrix.train'] 与 data['matrix.test'] 转换成 Pandas.DataFrame 格式，保存到 df['train'] 和 df['test'] 中（df 为字典格式：String -> DataFrame）

df = {}
for tpart in ['train', 'test']:
    datadict = {}
    datadict['class'] = data['vec.{}'.format(tpart)]['class']
    datavec = np.array(data['vec.{}'.format(tpart)]['w2v.mean'])
    for col in range(vecsize):
        datadict[col]= datavec[:, col]

    df[tpart] = pd.DataFrame(data=datadict)
    print "See df[{}]".format(tpart)
    display(df[tpart])
    print "\n\n\n"

print "Step 3 Succeed."

# 繁琐点：研究如何把 CSR 矩阵中的数据规整好放到 DataFrame 中，并与 Class 一一对应

See df[train]


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,class
0,0.001558,0.004365,0.000632,-0.000906,-0.001915,0.000811,0.006636,-0.001692,0.005637,-0.000466,...,-0.001899,-0.000541,0.004313,-0.000462,-0.001008,-0.000984,0.004622,-0.002609,-0.004004,alt.atheism
1,0.010691,0.009380,-0.009168,0.010979,0.007527,-0.003309,0.009023,-0.002332,0.002502,0.008829,...,-0.003226,-0.004776,-0.007298,-0.003070,0.007739,-0.000366,-0.013523,-0.000971,0.006258,alt.atheism
2,0.002090,-0.001998,-0.000089,0.001491,0.005646,0.002953,-0.001491,-0.001427,-0.003030,-0.000553,...,0.002799,0.000497,0.002337,0.001937,0.001053,0.002941,-0.002940,0.001466,-0.000386,alt.atheism
3,0.000076,0.000125,-0.000037,-0.000057,0.000239,0.000134,-0.000201,0.000332,0.000120,0.000009,...,0.000079,0.000018,0.000269,0.000035,-0.000110,0.000312,0.000038,-0.000284,0.000304,alt.atheism
4,-0.000239,-0.010530,0.003361,-0.002400,0.000935,0.004078,-0.002748,0.000447,0.002624,-0.003132,...,-0.001138,-0.002781,-0.003174,-0.000826,-0.004788,0.004906,-0.001101,-0.002321,-0.000512,alt.atheism
5,0.007988,-0.008523,-0.004368,0.004699,-0.006735,-0.000686,0.007814,0.001201,0.000582,0.002367,...,0.012112,-0.006082,0.000047,-0.006985,-0.005690,-0.000927,-0.013666,-0.001552,0.000776,alt.atheism
6,0.001017,0.002849,0.000412,-0.000592,-0.001250,0.000530,0.004331,-0.001104,0.003679,-0.000304,...,-0.001240,-0.000353,0.002815,-0.000301,-0.000658,-0.000642,0.003016,-0.001703,-0.002613,alt.atheism
7,0.002898,0.008120,0.001176,-0.001686,-0.003562,0.001509,0.012343,-0.003147,0.010484,-0.000866,...,-0.003533,-0.001007,0.008023,-0.000859,-0.001876,-0.001829,0.008596,-0.004853,-0.007447,alt.atheism
8,0.000458,0.000756,-0.000226,-0.000343,0.001445,0.000807,-0.001215,0.002007,0.000728,0.000053,...,0.000477,0.000111,0.001627,0.000212,-0.000663,0.001887,0.000231,-0.001713,0.001839,alt.atheism
9,0.002285,0.003979,0.007639,-0.001576,0.005827,-0.004290,-0.003740,0.008013,-0.004117,-0.000487,...,0.003548,-0.004791,-0.002285,-0.000528,-0.000821,0.001260,-0.000939,0.000045,0.002052,alt.atheism






See df[test]


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,class
0,0.006436,0.012668,-0.006423,0.012146,-0.009215,0.010104,-0.009986,-0.020605,0.002306,0.004094,...,0.002015,-0.003159,-0.005440,0.000383,0.004267,-0.000555,0.002340,-0.017017,-0.001320,alt.atheism
1,-0.001852,-0.000492,-0.002445,0.001612,-0.002473,-0.002895,0.001595,0.003776,0.002293,-0.004447,...,0.004466,0.002744,0.006909,-0.001785,-0.000060,0.004103,-0.002436,-0.003875,0.003202,alt.atheism
2,0.011585,0.040615,-0.002875,-0.002462,-0.000154,0.005779,-0.007070,-0.001935,0.012515,0.007867,...,0.015966,0.006913,0.008613,0.015725,-0.002469,0.007676,0.001677,0.003002,-0.007814,alt.atheism
3,0.002550,0.007581,-0.017568,-0.016240,-0.009751,0.013828,0.006954,-0.004135,0.001608,-0.005963,...,-0.004554,-0.000695,0.000935,-0.013021,0.008819,0.016827,-0.022691,-0.019779,0.013651,alt.atheism
4,0.000861,0.000988,0.006937,-0.000489,0.000142,-0.007329,-0.007650,-0.001437,0.000337,0.004682,...,0.003909,0.003017,-0.001963,0.000312,-0.000122,0.000758,0.002760,0.007945,-0.006277,alt.atheism
5,0.005612,-0.000053,0.000127,-0.002191,-0.002917,-0.004066,-0.003493,-0.000499,-0.001816,-0.000505,...,0.001946,0.001607,-0.005175,0.000654,-0.000521,-0.000978,0.004319,0.000463,-0.001901,alt.atheism
6,-0.046184,-0.010707,0.001626,0.025211,-0.028096,-0.010113,-0.013898,0.005437,-0.017856,-0.018506,...,0.012469,-0.006621,0.013136,0.032820,0.043819,-0.038226,-0.018283,-0.035961,-0.034990,alt.atheism
7,0.042974,0.041786,-0.005965,0.005437,0.018267,-0.008106,-0.054144,-0.027542,-0.028693,0.030998,...,0.001961,-0.012742,-0.000501,0.008579,0.005206,-0.023067,0.006790,-0.019963,-0.018468,alt.atheism
8,0.001082,-0.001035,-0.000046,0.000772,0.002923,0.001529,-0.000772,-0.000739,-0.001569,-0.000286,...,0.001449,0.000257,0.001210,0.001003,0.000545,0.001522,-0.001522,0.000759,-0.000200,alt.atheism
9,0.006342,0.020287,-0.008261,0.008630,0.002690,0.006176,-0.019100,-0.017833,0.003067,0.006178,...,-0.010471,-0.010715,0.005324,0.002187,-0.001142,0.002997,0.007491,-0.016165,-0.007499,alt.atheism






Step 3 Succeed.


In [21]:
for tpart in ['train', 'test']:
    # write data in DataFrame into CSV
    csvpath = os.path.join(csvpath_root, "{}-w2v.csv".format(tpart))
    df[tpart].to_csv(csvpath, columns=df[tpart].columns)

In [22]:
# if wanna read data from CSV file

df = {}

for tpart in ['train', 'test']:
    csvpath = os.path.join(os.getcwd(), 'data_CSV', '{}-w2v.csv'.format(tpart))
    if os.path.exists(csvpath):
        df[tpart] = pd.DataFrame.from_csv(csvpath)
        df[tpart] = df[tpart].sample(frac=1)
        df[tpart].reset_index(drop=True, inplace=True)
        print "read {} successfully".format(tpart)
        display(df[tpart])

read train successfully


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,class
0,0.007547,0.008326,-0.004550,-0.000180,-0.000432,0.002256,0.001575,0.002732,0.000177,0.002600,...,0.000100,-0.000295,-0.002544,-0.000964,0.001478,-0.003657,0.002884,0.003231,0.001083,alt.atheism
1,0.003511,0.007560,0.010124,0.004649,-0.001608,0.008403,-0.010975,-0.006962,-0.003016,0.005858,...,0.003209,0.004647,-0.002129,0.003806,0.002670,0.009646,0.001277,-0.003296,-0.000813,sci.med
2,0.038772,0.017721,0.019594,0.034038,0.015677,-0.000917,-0.038250,0.009332,0.025341,-0.019385,...,0.023851,-0.001845,-0.023511,-0.015851,0.015249,0.030856,0.007126,-0.011198,-0.000700,misc.forsale
3,0.012878,0.014207,-0.007764,-0.000307,-0.000737,0.003849,0.002688,0.004661,0.000303,0.004436,...,0.000170,-0.000504,-0.004341,-0.001645,0.002521,-0.006240,0.004921,0.005513,0.001847,sci.med
4,-0.005529,0.001529,0.001772,-0.001450,-0.006363,0.000434,-0.000010,-0.002773,0.002404,-0.000317,...,0.002711,-0.000096,0.003849,0.000143,-0.003007,0.001312,0.005110,0.000180,-0.004039,sci.med
5,0.006440,0.018044,0.002612,-0.003746,-0.007915,0.003354,0.027428,-0.006994,0.023298,-0.001925,...,-0.007851,-0.002238,0.017829,-0.001908,-0.004168,-0.004065,0.019103,-0.010785,-0.016550,misc.forsale
6,-0.017939,0.004961,0.005751,-0.004703,-0.020644,0.001409,-0.000034,-0.008998,0.007800,-0.001028,...,0.008795,-0.000313,0.012490,0.000464,-0.009757,0.004256,0.016579,0.000585,-0.013106,sci.med
7,-0.007106,-0.032602,0.004557,-0.000096,-0.012904,0.003218,-0.014964,-0.015800,0.003228,-0.013156,...,-0.006390,0.002065,0.002941,-0.002396,-0.004203,0.023060,0.007312,-0.015011,0.009204,alt.atheism
8,0.005064,-0.005402,-0.002769,0.002979,-0.004269,-0.000435,0.004953,0.000761,0.000369,0.001500,...,0.007678,-0.003855,0.000030,-0.004427,-0.003607,-0.000587,-0.008662,-0.000984,0.000492,alt.atheism
9,0.003374,-0.003226,-0.000143,0.002407,0.009113,0.004766,-0.002406,-0.002303,-0.004891,-0.000892,...,0.004519,0.000803,0.003772,0.003127,0.001700,0.004747,-0.004745,0.002366,-0.000623,alt.atheism


read test successfully


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,class
0,-0.001702,0.000937,-0.000077,0.002912,-0.000136,-0.001481,0.005113,0.000089,0.006080,0.002718,...,0.000760,0.000347,0.000269,-0.001737,0.000279,-0.001366,0.002393,-0.000601,-0.002293,comp.os.ms-windows.misc
1,-0.005177,-0.004713,-0.001844,0.000361,-0.002035,-0.002277,0.000369,0.001379,-0.000537,-0.005652,...,0.001101,-0.002052,-0.001523,0.001181,-0.000369,0.001698,-0.000445,0.000090,-0.002788,sci.med
2,-0.036731,-0.020624,-0.003409,-0.004905,-0.006086,0.019713,-0.000442,0.018946,0.003761,-0.038462,...,-0.003518,-0.006121,-0.006401,-0.006948,-0.003729,0.006214,0.004336,-0.022368,0.001565,sci.med
3,0.004081,-0.003902,-0.000173,0.002912,0.011022,0.005765,-0.002911,-0.002786,-0.005916,-0.001079,...,0.005465,0.000971,0.004563,0.003783,0.002056,0.005741,-0.005739,0.002862,-0.000754,comp.os.ms-windows.misc
4,0.000758,0.001974,0.000337,-0.002142,0.002769,0.001602,0.000185,0.000110,-0.001621,-0.002550,...,-0.002075,0.002741,0.000169,0.006843,0.003054,-0.002733,-0.002750,0.000899,0.000946,alt.atheism
5,0.008301,-0.005105,0.004769,0.007776,0.001584,0.005775,-0.009499,-0.003145,-0.000377,-0.005323,...,0.011485,0.009998,0.004619,0.006697,0.003472,0.006058,-0.000156,0.002038,-0.009749,alt.atheism
6,0.036561,-0.017230,-0.040565,-0.003307,-0.001809,-0.021638,0.000308,0.001480,0.011715,-0.002635,...,0.018986,0.003047,-0.007743,0.003577,0.009418,0.002895,0.006465,-0.015753,-0.011666,misc.forsale
7,0.004483,-0.002757,0.002576,0.004200,0.000855,0.003119,-0.005130,-0.001698,-0.000203,-0.002875,...,0.006203,0.005400,0.002495,0.003617,0.001875,0.003272,-0.000084,0.001101,-0.005265,alt.atheism
8,0.017853,-0.017070,-0.000756,0.012740,0.048223,0.025221,-0.012734,-0.012187,-0.025883,-0.004720,...,0.023911,0.004247,0.019962,0.016549,0.008995,0.025118,-0.025108,0.012519,-0.003298,comp.os.ms-windows.misc
9,-0.000152,0.000411,-0.000485,-0.000363,0.000117,-0.000004,-0.000062,-0.000481,0.000140,0.000161,...,-0.000779,-0.000254,-0.000067,-0.000073,0.000290,0.000375,0.000306,0.000368,0.000169,sci.med


In [23]:
from sklearn.preprocessing import LabelBinarizer

csvpath_root = os.path.join(os.getcwd(), 'data_CSV')

df_new = {}

lb = LabelBinarizer()
lb.fit(df['train']['class'])

for tpart in ['train', 'test']:
    labels = lb.transform(df[tpart]['class'])
    labelsDf = pd.DataFrame(labels, columns=["class-{}".format(i) for i in range(len(lb.classes_))])
    df_new[tpart] = {}
    df_new[tpart]['y'] = labelsDf
    df_new[tpart]['x'] = df[tpart].drop('class', axis=1)
    df_new[tpart]['all'] = df_new[tpart]['x'].join(df_new[tpart]['y'])
    #save in CSV
    for subpart in ['x', 'y', 'all']:
        csvpath = os.path.join(csvpath_root, "{}-w2v-cleanLabels-{}.csv".format(tpart, subpart))
        df_new[tpart][subpart].to_csv(csvpath)
    
print "label cleaning succussfully"

label cleaning succussfully


In [25]:
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

sess = tf.InteractiveSession()

COL_OUTCOME = 'class'
COL_FEATURE = [col for col in list(df['train'].columns) if col != COL_OUTCOME]

# cls2num = {cls:ind for (ind, cls) in enumerate(df['train']['class'].unique())}

count_feature = len(COL_FEATURE)
count_class = len(df['train']['class'].unique())

x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
y_ = tf.placeholder(tf.float32, shape=[None, count_class], name='y_')

W = tf.Variable(tf.zeros([count_feature, count_class]))
b = tf.Variable(tf.zeros([count_class]))
y = tf.matmul(x, W) + b

# cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_text = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(conv2d(x_text, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32, name='keep_prob')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, count_class])
b_fc2 = bias_variable([count_class])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

print "CNN initialization finished"

CNN initialization finished


In [55]:
### Start to traini and evaluate the model

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.global_variables_initializer())

x_input = df_new['train']['x']
x_input = [np.array([
            np.float32(x_input.iloc[i].values)
        ])
    for i in range(x_input.shape[0])]
y_input = df_new['train']['y']
y_input = [np.array([
            np.float32(y_input.iloc[i].values)
        ])
    for i in range(y_input.shape[0])]
# y_input = [np.array([y_input.iloc[i].values]) for i in range(y_input.shape[0])]

# not use random input

for i in range(df['train'].shape[0] - 50):
    if 0 == i % 100:
        train_accuracy = []
        for j in range(50):
            train_accuracy.append(accuracy.eval(feed_dict={
                    keep_prob: 1,
                    x:  np.array([elem[0] for elem in x_input[i+j:i+j+50]]),#x_input.iloc[i+j].values, #
                    y_: np.array([elem[0] for elem in y_input[i+j:i+j+50]])#y_input.iloc[i+j].values #
                })
            )
        print "step {}, training accuracy {}".format(i, np.mean(train_accuracy))
    train_step.run(feed_dict={
        keep_prob: 0.5,
        x:  np.array([elem[0] for elem in x_input[i:i+50]]),#x_input.iloc[i].values, #
        y_: np.array([elem[0] for elem in y_input[i:i+50]])#y_input.iloc[i].values#
    })

print "CNN training finished"

step 0, training accuracy 0.268000006676
step 100, training accuracy 0.335599988699
step 200, training accuracy 0.352800011635
step 300, training accuracy 0.312799990177
step 400, training accuracy 0.322399973869
step 500, training accuracy 0.358400046825
step 600, training accuracy 0.303599983454
step 700, training accuracy 0.374799966812
step 800, training accuracy 0.382800012827
step 900, training accuracy 0.331200003624
step 1000, training accuracy 0.284799993038
step 1100, training accuracy 0.322400003672
step 1200, training accuracy 0.32959997654
step 1300, training accuracy 0.464000016451
step 1400, training accuracy 0.440400004387
step 1500, training accuracy 0.374399989843
step 1600, training accuracy 0.371200025082
step 1700, training accuracy 0.300800025463
step 1800, training accuracy 0.438400030136
step 1900, training accuracy 0.37760001421
step 2000, training accuracy 0.493200004101
step 2100, training accuracy 0.382799983025
CNN training finished


In [57]:
x_input = df_new['test']['x']
x_input = [np.array([
            np.float32(x_input.iloc[i].values)
        ])
    for i in range(x_input.shape[0])]
y_input = df_new['test']['y']
y_input = [np.array([
            np.float32(y_input.iloc[i].values)
        ])
    for i in range(y_input.shape[0])]

for i in range(df['test'].shape[0] - 50):
    if 0 == i % 100:
        train_accuracy = []
        for j in range(50):
            train_accuracy.append(accuracy.eval(feed_dict={
                    keep_prob: 1,
                    x:  np.array([elem[0] for elem in x_input[i+j:i+j+50]]),#x_input.iloc[i+j].values, #
                    y_: np.array([elem[0] for elem in y_input[i+j:i+j+50]])#y_input.iloc[i+j].values #
                })
            )
        print "step {}, testing accuracy {}".format(i, np.mean(train_accuracy))

        
print "CNN testing finished"

step 0, testing accuracy 0.444799989462
step 100, testing accuracy 0.447999984026
step 200, testing accuracy 0.392399966717
step 300, testing accuracy 0.427600026131
step 400, testing accuracy 0.335999935865
step 500, testing accuracy 0.322400003672
step 600, testing accuracy 0.461600005627
step 700, testing accuracy 0.444399982691
step 800, testing accuracy 0.345999985933
step 900, testing accuracy 0.403599977493
step 1000, testing accuracy 0.403600007296
step 1100, testing accuracy 0.352399975061
step 1200, testing accuracy 0.400400012732
step 1300, testing accuracy 0.28560000658
step 1400, testing accuracy 0.35799998045
CNN testing finished


In [70]:
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier

X_train = df['train'].drop('class', axis=1)
y_train = df['train']['class']

# clf = KNeighborsClassifier()
clf = LinearSVC()
# clf = SGDClassifier()
# clf = GaussianNB()
# clf = DecisionTreeClassifier()
# clf =MultinomialNB()
# clf = MLPClassifier()
clf.fit(X_train, y_train)
# print "Training fishied with clf with [n_classes, n_features]: {}".format(clf.coef_)
print "Step 4 finished"

Step 4 finished


In [71]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

X_test = df['test'].drop('class', axis=1)
y_test_true = df['test']['class']

y_test_pred = clf.predict(X_test)
print accuracy_score(y_test_true, y_test_pred)
print f1_score(y_test_true, y_test_pred, average='macro')
print f1_score(y_test_true, y_test_pred, average='micro')

0.420947298199
0.414043038024
0.420947298199
